In [ ]:
# default_exp __init__

# Core

> Team-strength models in Python

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
import json
import datetime as dt
import numpy as np
import pprint

In [ ]:
#export

# For now, just re-export everything
from mezzala.adapters import *
from mezzala.blocks import *
from mezzala.models import *
from mezzala.weights import *
from mezzala.parameters import *

Let's demo

In [ ]:
with open('../data/premier-league-1516.json', 'r') as f:
    pl_1516 = json.load(f)

# Let's parse the dates, too
for match in pl_1516:
    match['date'] = dt.datetime.strptime(match['date'], '%Y-%m-%d')
    
pl_1516[0:3]

[{'date': datetime.datetime(2015, 8, 8, 0, 0),
  'team1': 'Manchester United FC',
  'team2': 'Tottenham Hotspur FC',
  'score': {'ft': [1, 0]}},
 {'date': datetime.datetime(2015, 8, 8, 0, 0),
  'team1': 'AFC Bournemouth',
  'team2': 'Aston Villa FC',
  'score': {'ft': [0, 1]}},
 {'date': datetime.datetime(2015, 8, 8, 0, 0),
  'team1': 'Leicester City FC',
  'team2': 'Sunderland AFC',
  'score': {'ft': [4, 2]}}]

A model in `mezzala` is composed of 2 parts:

* Model blocks (see `mezzala.blocks`)
* An adapter (see `mezzala.adapters`)

The model blocks determine which terms your model estimates. In general, you will want
to estimate offensive and defensive strength for each team (`TeamStrength`) and 
as well as home advantage `HomeAdvantage`.

The selected model blocks can be supplied to the model as a list:

In [ ]:
blocks = [TeamStrength(), HomeAdvantage()]

An adapter connects your model to the data source. In other words, it tells the model how find the information needed to fit.

The information needed is determined by which model blocks are used. In our case,

* All models require `home_goals` and `away_goals`
* `TeamStrength` - requires `home_team` and `away_team`

`HomeAdvantage` doesn't require any information, since it assumes all matches have equal home-field advantage by default.

In [ ]:
adapter = KeyAdapter(               # `KeyAdapter` = data['...']
    home_team='team1',
    away_team='team2',
    home_goals=['score', 'ft', 0],  # Get nested fields with lists of fields
    away_goals=['score', 'ft', 1],  # i.e. data['score']['ft'][1]
)

Pulling this together, we can construct a model from an adapter and blocks

In [ ]:
model = DixonColes(adapter=adapter, blocks=blocks)
model.fit(pl_1516)

# All estimates should be valid numbers
assert all(not np.isnan(x) for x in model.params.values())

# Home advantage should be positive
assert 1.0 < np.exp(model.params[HFA_KEY]) < 2.0

Let's inspect the parameters a bit. First, let's look at the boring (non-team) ones:

In [ ]:
param_keys = model.params.keys()
param_key_len = max(len(str(k)) for k in param_keys)

for k in param_keys:
    if not isinstance(k, TeamParameterKey):
        key_str = str(k).ljust(param_key_len + 1)
        print(f'{key_str}: {np.exp(model.params[k]):0.2f}')

ParameterKey(label='Home-field advantage')           : 1.23
ParameterKey(label='Rho')                            : 0.94


And the team ones. Let's look at each team's attacking quality:

In [ ]:
teams = {k.label for k in param_keys if isinstance(k, TeamParameterKey)}

team_offence = [(t, np.exp(model.params[OffenceParameterKey(t)])) for t in teams]
for team, estimate in sorted(team_offence, key=lambda x: -x[1]):
    print(f'{team}: {estimate:0.2f}')

Manchester City FC: 1.38
Tottenham Hotspur FC: 1.33
Leicester City FC: 1.31
West Ham United FC: 1.27
Arsenal FC: 1.25
Liverpool FC: 1.23
Everton FC: 1.16
Chelsea FC: 1.15
Southampton FC: 1.14
Manchester United FC: 0.94
Sunderland AFC: 0.94
AFC Bournemouth: 0.89
Newcastle United FC: 0.87
Swansea City FC: 0.82
Stoke City FC: 0.81
Watford FC: 0.78
Norwich City FC: 0.77
Crystal Palace FC: 0.76
West Bromwich Albion FC: 0.66
Aston Villa FC: 0.54


In [ ]:
team_defence = [(t, np.exp(model.params[DefenceParameterKey(t)])) for t in teams]
for team, estimate in sorted(team_defence, key=lambda x: x[1]):
    print(f'{team}: {estimate:0.2f}')

Manchester United FC: 0.82
Tottenham Hotspur FC: 0.84
Leicester City FC: 0.86
Arsenal FC: 0.86
Southampton FC: 0.97
Manchester City FC: 0.99
West Bromwich Albion FC: 1.10
Watford FC: 1.17
Liverpool FC: 1.19
Crystal Palace FC: 1.19
Swansea City FC: 1.21
West Ham United FC: 1.22
Chelsea FC: 1.26
Stoke City FC: 1.28
Everton FC: 1.32
Sunderland AFC: 1.46
Newcastle United FC: 1.52
Norwich City FC: 1.55
AFC Bournemouth: 1.57
Aston Villa FC: 1.75


Making predictions for a single match

In [ ]:
scorelines = model.predict_one({
    'team1': 'Manchester City FC',
    'team2': 'Swansea City FC',
})

# Probabilities should sum to 1
assert np.isclose(
    sum(p.probability for p in scorelines),
    1.0
)

scorelines[0:5]

[ScorelinePrediction(home_goals=0, away_goals=0, probability=0.05607098119573087),
 ScorelinePrediction(home_goals=0, away_goals=1, probability=0.045632172429469826),
 ScorelinePrediction(home_goals=0, away_goals=2, probability=0.01856838525229347),
 ScorelinePrediction(home_goals=0, away_goals=3, probability=0.005037161466295112),
 ScorelinePrediction(home_goals=0, away_goals=4, probability=0.0010248466126474723)]

In [ ]:
outcomes = scorelines_to_outcomes(scorelines)

# MCFC should have a better chance of beating Swansea
# at home than Swansea do of winning away
assert outcomes[Outcomes('Home win')].probability > outcomes[Outcomes('Away win')].probability

list(outcomes.values())

Or for multiple matches

In [ ]:
many_scorelines = model.predict([
    {'team1': 'Manchester City FC',
     'team2': 'Swansea City FC'},
    {'team1': 'Manchester City FC',
     'team2': 'West Ham United FC'}
])

What about a model with a different weighting method?

By default, the `DixonColes` model weights all matches equally. However, it's more realistic to give matches
closer to the current date a bigger weight than those a long time ago.

The original Dixon-Coles paper suggests using an exponential weight, and we can use the same:

In [ ]:
season_end_date = max(match['date'] for match in pl_1516)

weight = ExponentialWeight(
    # Value of `epsilon` is taken from the original paper
    epsilon=-0.0065,  
    key=lambda x: (season_end_date - x['date']).days
)

In [ ]:
model_exp = DixonColes(
    adapter=adapter,
    blocks=blocks,
    weight=weight
)
model_exp.fit(pl_1516)

How much does that change the ratings at season-end?

In [ ]:
for k in param_keys:
    key_str = str(k).ljust(param_key_len + 1)
    model_param = np.exp(model.params[k])
    model_exp_param = np.exp(model_exp.params[k])
    print(f'{key_str}: {model_param:0.2f} -> {model_exp_param:0.2f} ({model_exp_param/model_param:0.2f})')